<div style="font-size: 32px; color: gray; font-weight: bold; text-align: center;">Get Data for Stock Analysis</div>

---

## Libraries

In [2]:
# !pip install yfinance

In [3]:
# Imports
import yfinance as yf
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
# import plotly
# import plotly.express as px
# import plotly.graph_objects as go
# pd.options.plotting.backend = "plotly"
import numpy as np
import boto3
from io import StringIO
from sagemaker import get_execution_role
import warnings
warnings.filterwarnings("ignore")
import pickle

## <span style="color: red">Parameters</span>

In [3]:
# Update the following
# cob_date = "2021-05-14" # use this as the row index *** NOT USED?? ***
prev_yr_end = "2020-12-31"

# Details to capture from info function for metadata and ratio calculations:

metadata = [
    "longName",
    "sector",
    "industry",
]
    
static_ratios = [
    # ratios:
#     "trailingPE",
#     "forwardPE",
#     "priceToSalesTrailing12Months",
    "profitMargins",
    "payoutRatio",
    "sharesOutstanding",
    "trailingEps",
    "forwardEps",
]

financials = [
    "Total Revenue",
]

cashflow = [
    "Total Cash From Operating Activities",
]

balsheet = [
    "Total Liab",
    "Total Stockholder Equity",
    "Total Current Assets",
    "Total Current Liabilities",
]

# Type translation
# cob_date = pd.to_datetime(cob_date) #.date() # is this the right date format?
prev_yr_end = pd.to_datetime(prev_yr_end) #.date() # is this the right date format?

In [4]:
# print(cob_date, type(cob_date))
print(prev_yr_end, type(prev_yr_end))

2020-12-31 00:00:00 <class 'pandas._libs.tslibs.timestamps.Timestamp'>


## Investment Ratios

### Price-earnings ratio

Divide a company's share price by its annual earnings per share to calculate the P/E ratio. This ratio shows how much investors are willing to pay for $1 of a company's earnings. "It is probably the best way of comparing assets in different sectors and of finding true bargains," says Steven Jon Kaplan, CEO of True Contrarian Investments. Higher P/E ratios suggest a company's future earnings are expected to grow and may appear overvalued compared with companies with lower P/Es. That said, a high or low P/E doesn't necessarily indicate a good or bad investment; it offers a snapshot that begs additional inquiry. Given the uncertainty of future cash flows, it can be helpful for investors to rely on historical P/Es and use a mix of other ratios to evaluate and pick stocks.

Calc:

- Adj Close / trailingEps
- Adj Close / forwardEps

NB: can compare across industries/sectors

### Price-sales ratio

Robert Johnson, professor of finance at Creighton University in Omaha, Nebraska, touts the benefits of calculating a stock's price divided by sales per share, commonly referred to as the price-to-sales ratio. "The price-to-sales ratio is used by analysts who want to eliminate some of the distortions that can result in company earnings," Johnson says. It's a useful ratio to determine whether a company has earnings, cash flow or even positive book value since sales is always a positive number. A lower ratio suggests you've found a bargain, or a value stock. Industry consensus says lower P/S stocks have better value because investors are paying less for every dollar of a company's revenues. P/S ratio values can vary across sectors, so to best assess a company's P/S, compare it with industry peers.

In [5]:
# Calc:

# a. Total Revenue

# b. sharesOutstanding

# c. Adj Close / (a/b)

# NB: compare to peers in same industry

### Profit margin ratio

This is the amount of profit a company makes for every unit of sales. Investors calculate this ratio by dividing net profit over revenue. Profit margins are unique to an industry – with grocery chains known for low profit margins, while software companies can claim double-digit ratios. But this information doesn't necessarily mean that it's better to buy a software company than a grocery store stock. A high profit margin means a business can offer products priced higher than its costs, yielding profits through effective pricing strategies. A low profit margin may mean there are inefficient pricing strategies, where a business cannot produce enough profit to cover expenses. Any stock could be a winner with a growing revenue stream and steady profit margins.

Calc:
- profitMargins

NB: compare to peers in same industry. Low profit margin could be ok if revenue growing and profit margin steady.

### Dividend payout ratio

Companies with rising dividend payments are favored by John Robinson, owner and founder of Nest Egg Guru in Hawaii. The dividend payout ratio is the percentage of net income paid to investors in the form of dividends. This ratio tells investors how much earnings are paid out in dividends versus how much is reinvested back into the company. The higher the percentage, the less money remains to reinvest back into growing the company. "Companies that pay out less than 60% of their earnings as dividends tend to have room for further dividend increases and the ability to withstand temporary earnings downturns without having to reduce or eliminate dividend payments," he says.

Calc:
- payoutRatio

NB: amount of earnings paid out as dividend. Rest is retained / reinvested. Lower % may indicate growth or ability to withstand earnings shocks.

### Price-free cash flow ratio

Tim Parker, a partner at Regency Wealth Management in New York City, reveres free cash flow because that is the amount of money left over after a company reinvests in the business to pay dividends, buy back shares or make acquisitions. To determine price-free cash flow, divide the company's share price by the operating free cash flow per share. The ratio measures how much cash a company earns for each share of stock. Investors want to search for companies with growing free cash flow that are selling at a bargain. Parker favors this ratio since free cash flow is harder to manipulate than earnings. A lower ratio indicates a company may be undervalued, while a higher ratio may signal overvaluation.

In [6]:
# Calc:

# 1. Operating cashflow for previous year end:
# su = yf.Ticker("SU.PA")
# su.cashflow[pd.to_datetime('2020-12-31')].loc["Total Cash From Operating Activities"]
# PLUS
# su.cashflow[pd.to_datetime('2020-12-31')].loc["Capital Expenditures"]

# 2. Divided by
# sharesOutstanding 

# 3. Equals: Operating Free Cashflow per Share

# 4. Finally: Adj Close / Operating Free Cashflow per Share

# NB: this ratio focusses on cash rather than earnings (cash is harder to manipulate). 
# A lower ratio may indicate company undervalued and high ratio may indicate overvalued.


### Debit-equity ratio

Valuation ratios are important, but so are quality measures, such as debt and liquidity metrics. Divide a company's total liabilities by its shareholder equity to compute the debt-equity ratio. This ratio explains a company's financial leverage, the comparison between borrowed funds and equity or ownership. Think of this ratio like a homeowner's mortgage value versus principal on the home. A greater proportion of debt constrains a company's flexibility to grow as more revenue is directed to pay debt costs. Like most ratios, compare the debt-equity ratio to those of other industry members, as some sectors, such as utilities, have higher typical debt ratios compared with others.

In [7]:
# Calc:
# su = yf.Ticker("SU.PA")
# tot_liab = su.balance_sheet[pd.to_datetime('2020-12-31')].loc["Total Liab"]
# tot_se = su.balance_sheet[pd.to_datetime('2020-12-31')].loc["Total Stockholder Equity"]
# de_ratio = tot_liab/tot_se
# print(tot_liab)
# print(tot_se)
# print(de_ratio)

# NB: high ratio (leverage) may indicate has less flexibility to grow as it is constrained by its debt costs.
# Compare to other companies in same industry as different sectors have different typical leverage. 
# SEEMS LIKE THERE ARE MULTIPLE WAYS TO IMPLEMENT. CONSISTENCY IS MOST IMPORTANT.

### Current ratio

Sameer Samana, global equity and technical strategist at Wells Fargo Investment Institute in St. Louis, recommends examining the quick ratio and current ratio. These liquidity ratios measure if a company has enough working capital to handle potential downturns and financial setbacks. The current ratio divides current assets by current liabilities to measure how much cash a company has on hand to pay short-term obligations within a year. The quick ratio sums cash, marketable securities and accounts receivables and divides this sum by current liabilities. Higher numbers for these ratios suggest greater liquidity, while lower ratios may suggest a company cannot meet short-term obligations.

In [8]:
# Calc:
# su = yf.Ticker("SU.PA")
# ca = su.balance_sheet[pd.to_datetime('2020-12-31')].loc["Total Current Assets"]
# cl = su.balance_sheet[pd.to_datetime('2020-12-31')].loc["Total Current Liabilities"]
# current_ratio = ca/cl
# print(ca)
# print(cl)
# print(current_ratio)

# NB: working capital indicates ability to handle potential financial downturns and setbacks.

### EBITDA-to-sales ratio

This metric is the company's EBITDA – which is an abbreviation for earnings before interest, taxes, depreciation and amortization – divided by its net sales. This ratio is used to evaluate a company's overall profitability or earnings before expenses, by comparing revenue with earnings. "The stability of EBITDA typically determines investors' appetite for the amount of debt it believes the business should have," says Bryan Lee, chief investment officer at Blue Zone Wealth Advisors in Los Angeles. "A more levered company brings higher volatility for how the equity trades. This volatility can amplify returns on the upside but also to the downside," he says. EBITDA margin offers a transparent view into business operations by eliminating noncash or nonoperating expenses like interest costs, taxes and depreciation that may dim profits, giving a more precise view of a company's profitability.

In [9]:
# Calc:

# su = yf.Ticker("SU.PA")
# su.financials
# su.cashflow[pd.to_datetime('2020-12-31')].loc["Total Cash From Operating Activities"]

# NB: don't see how to get EBITDA from YF

## Functions

In [10]:
# Function to save a .csv in S3 from a DataFrame of our SageMaker python notebook
def _write_dataframe_to_csv_on_s3(DESTINATION, dataframe, filename):
    """ Write a dataframe to a CSV on S3 """
    print(
        "Writing {} records to '{}' in bucket '{}'.".format(
            len(dataframe), filename, DESTINATION
        )
    )
    # Create buffer
    csv_buffer = StringIO()
    # Write dataframe to buffer
    dataframe.to_csv(csv_buffer, sep=",", index=True)
    # Create S3 object
    s3_resource = boto3.resource("s3")
    # Write buffer to S3 object
    s3_resource.Object(DESTINATION, filename).put(Body=csv_buffer.getvalue())

## Fetch data

### Get ticker info

In [11]:
mystock = "SU.PA" # ticker code per Yahoo Finance
myticker = yf.Ticker(mystock)

In [12]:
info_ticker = myticker.info
for info in info_ticker:
    print("-" * 72)
    print(info, ":")
    print(info_ticker[info])

------------------------------------------------------------------------
zip :
92500
------------------------------------------------------------------------
sector :
Industrials
------------------------------------------------------------------------
fullTimeEmployees :
135000
------------------------------------------------------------------------
longBusinessSummary :
Schneider Electric S.E. provides energy and automation digital solutions worldwide. The company operates in two segments, Energy Management and Industrial Automation. It offers busway and cable management products, circuit breakers and switches, contactors and protection relays, electrical protection and control products, fuse and safety switches, motor starters and protection components, power monitoring and control products, power quality and power factor correction products, pushbuttons, switches, pilot lights and joysticks, software products, surge protection and power conditioning products, and switchboards and en

In [13]:
# for key in myticker.info:
#     print(key)

# print(myticker.info["longName"])
# print(myticker.info["sector"])
# print(myticker.info["industry"])

In [14]:
# mytickers = yf.Tickers(["MC.PA", "RDSA.AS"])

In [15]:
# for ticker in mytickers.tickers:
#     print(mytickers.tickers[ticker].info["longName"])

### Parameters for the history function

### Get history of a stock

In [16]:
# hist = myticker.history(period="1y") # check docs for parameters

In [17]:
# hist.head()

In [18]:
# hist["Close"].plot(figsize=(16,9), grid=True, title=f"{mystock}\n{myticker.info['longName']}") # without Plotly backend

In [6]:
su = yf.Ticker("SU.PA")
su.dividends #[pd.to_datetime('2020-12-31')].loc["Total Liab"]

Date
2000-06-09    0.662030
2001-05-07    0.790483
2002-05-31    0.642268
2003-05-20    0.494052
2004-05-10    0.815186
2005-05-17    0.889294
2006-05-09    1.111617
2007-02-22    0.138335
2007-05-02    1.500000
2008-04-25    1.650000
2009-05-04    1.725000
2010-05-04    1.025000
2011-04-29    1.600000
2012-05-11    1.700000
2013-05-02    1.870000
2014-05-14    0.422000
2015-04-30    1.360000
2016-05-05    2.000000
2017-05-08    2.040000
2018-05-02    2.200000
2019-04-30    2.350000
2020-05-05    2.550000
2021-05-10    2.600000
Name: Dividends, dtype: float64

In [7]:
su.earnings

,Revenue,Earnings
Year,,
2017,24743000000,2150000000
2018,25720000000,2334000000
2019,27158000000,2413000000
2020,25159000000,2126000000


In [10]:
su.actions

,Dividends,Stock Splits
Date,,
2000-06-09,0.662030,0.0
2001-05-07,0.790483,0.0
2002-05-31,0.642268,0.0
2003-05-20,0.494052,0.0
2004-05-10,0.815186,0.0
2005-05-17,0.889294,0.0
2006-05-09,1.111617,0.0
2007-02-22,0.138335,0.0
2007-05-02,1.500000,0.0


In [11]:
su.cashflow

,2020-12-31,2019-12-31,2018-12-31,2017-12-31
Investments,1.100000e+07,5.900000e+07,-3.100000e+07,2.600000e+07
Change To Liabilities,3.440000e+08,-4.100000e+07,-9.800000e+07,3.040000e+08
Total Cashflows From Investing Activities,-3.250000e+09,-9.160000e+08,-1.705000e+09,-1.238000e+09
Net Borrowings,2.976000e+09,-6.140000e+08,2.110000e+08,-1.740000e+08
Total Cash From Financing Activities,2.585000e+09,-2.125000e+09,-1.757000e+09,-1.522000e+09
Change To Operating Activities,2.670000e+08,8.000000e+07,-9.700000e+07,4.700000e+07
Issuance Of Stock,4.300000e+07,1.680000e+08,1.640000e+08,1.610000e+08
Net Income,2.126000e+09,2.413000e+09,2.334000e+09,2.150000e+09
Change In Cash,3.367000e+09,1.164000e+09,-5.360000e+08,2.370000e+08
Repurchase Of Stock,-5.000000e+07,-2.660000e+08,-8.290000e+08,-1.710000e+08


In [14]:
-1.413000e+09/2126000000

-0.6646284101599247

### Fetch historical prices, volumes etc.

In [19]:
# Stocks comprising the euronext 100
euronext100 = ['KBC.BR', 'JMT.LS', 'BNP.PA', 'BN.PA', 'GBLB.BR', 'KN.PA', 'AGS.BR', 'LR.PA', 'MC.PA', 'ABI.BR', 'EDEN.PA', 'RDSA.AS', 'OR.PA', 'CAP.PA', 'SAN.PA', 'GALP.LS', 'SW.PA', 'INGA.AS', 'KER.PA', 'PHIA.AS', 'ATO.PA', 'ILD.PA', 'COV.PA', 'AIR.PA', 'AC.PA', 'TEP.PA', 'UBI.PA', 'ASML.AS', 'RAND.AS', 'VIE.PA']

# All stocks on Euronext Paris, Amsterdam & Brussels
euronext_pab = ['2CRSI.PA', '2MX.PA', 'ASP.PA', 'AALB.AS', 'ABI.BR', 'AB.PA', 'ABCA.PA', 'ABEO.PA', 'ABNX.PA', 'ABVX.PA', 'ABN.AS', 'ABO.BR', 'ACPH.BR', 'ACAN.PA', 'ACCEL.AS', 'ACCB.BR', 'AC.PA', 'ACNV.PA', 'AXS.AS', 'ACKB.BR', 'EOS.PA', 'ATI.PA', 'ALDV.PA', 'ALP.PA', 'ADOC.PA', 'ADP.PA', 'ADUX.PA', 'ADV.PA', 'ADVIC.PA', 'ADVI.PA', 'ADYEN.AS', 'AED.BR', 'AGN.AS', 'AKOM.PA', 'AGS.BR', 'AGFB.BR', 'AD.AS', 'AF.PA', 'AI.PA', 'AIR.PA', 'AJAX.AS', 'AKA.PA', 'AKW.PA', 'AKZA.AS', 'AAA.PA', 'ABIO.PA', 'ALD.PA', 'ALFEN.AS', 'CDANV.PA', 'CDA.PA', 'ALM.PA', 'ALO.PA', 'ALONV.PA', 'LTA.PA', 'ALTA.PA', 'AREIT.PA', 'ATE.PA', 'ALTUR.PA', 'ALX.AS', 'AMG.AS', 'AMPLI.PA', 'ACOMO.AS', 'AMUN.PA', 'APAM.AS', 'APM.PA', 'ARCAD.AS', 'MT.AS', 'ARG.PA', 'ARGX.BR', 'AKE.PA', 'ARTE.PA', 'PRC.PA', 'ARTO.PA', 'ASC.BR', 'ASIT.BR', 'ASM.AS', 'ASML.AS', 'ASRNL.AS', 'ASY.PA', 'ATA.PA', 'ATEME.PA', 'ATEB.BR', 'ATO.PA', 'ATRS.AS', 'AUB.PA', 'AUGR.PA', 'AURE.PA', 'AURS.PA', 'AVTX.AS', 'AVT.PA', 'CS.PA', 'CSNV.PA', 'AXW.PA', 'BSGR.AS', 'BCRA.PA', 'BAIN.PA', 'BALTA.BR', 'BALYO.PA', 'BAMNB.AS', 'BANI.BR', 'BUI.PA', 'BAR.BR', 'BFIT.AS', 'BASS.PA', 'BLC.PA', 'BESI.AS', 'BEFB.BR', 'BEKB.BR', 'FBEL.PA', 'BELR.BR', 'BELU.BR', 'BEN.PA', 'BBED.AS', 'BEVER.AS', 'BB.PA', 'BIG.PA', 'BCART.BR', 'BIM.PA', 'BLEE.PA', 'BNP.PA', 'BOI.PA', 'BOL.PA', 'BOLNV.PA', 'BON.PA', 'BOTHE.BR', 'BOOST.PA', 'BOKA.AS', 'BSD.PA', 'BGHL.AS', 'BGHS.AS', 'EN.PA', 'ENNV.PA', 'BPOST.BR', 'BNB.BR', 'BREB.BR', 'BRILL.AS', 'BRNL.AS', 'BVI.PA', 'BUR.PA', 'CAT31.PA', 'CAFO.PA', 'CBDG.PA', 'CAMB.BR', 'CAPLI.PA', 'CAP.PA', 'CPINV.BR', 'CARM.PA', 'CARP.PA', 'CA.PA', 'CO.PA', 'CAS.PA', 'CATG.PA', 'CTRG.PA', 'CATR.PA', 'CBOT.PA', 'CGM.PA', 'CGR.PA', 'CYAD.BR', 'CENER.BR', 'CFEB.BR', 'CFI.PA', 'CGG.PA', 'CRI.PA', 'CHSR.PA', 'CDI.PA', 'CIB.PA', 'COMB.BR', 'CLA.PA', 'CMCOM.AS', 'COM.PA', 'CNV.PA', 'CNP.PA', 'CCEP.AS', 'COFA.PA', 'COFB.BR', 'COH.PA', 'RE.PA', 'COLR.BR', 'CRBN.AS', 'CLB.AS', 'COUR.PA', 'COV.PA', 'COVH.PA', 'CRAP.PA', 'CRAV.PA', 'CRBP2.PA', 'CIV.PA', 'CRLA.PA', 'CRLO.PA', 'CMO.PA', 'CNF.PA', 'CCN.PA', 'CAF.PA', 'CRSU.PA', 'CRTO.PA', 'ACA.PA', 'OPTI.BR', 'CROS.PA', 'SX.PA', 'CTAC.AS', 'CTPNV.AS', 'CLEX.BR', 'DIE.BR', 'BN.PA', 'AM.PA', 'DSYNV.PA', 'DSY.PA', 'DBV.PA', 'DECB.BR', 'DLTA.PA', 'DBG.PA', 'DVT.PA', 'DGB.AS', 'DGE.PA', 'DGM.PA', 'DPAM.PA', 'DPA.AS', 'DSC2.AS', 'DSC2S.AS', 'DSM.AS', 'EAS2P.AS', 'ECONB.BR', 'EDEN.PA', 'EDF.PA', 'EFIC1.AS', 'EFICT.AS', 'GID.PA', 'FGR.PA', 'EKI.PA', 'EKOP.BR', 'ELEC.PA', 'EEM.PA', 'ELI.BR', 'ELIOR.PA', 'ELIS.PA', 'ENGI.PA', 'EPS.PA', 'ENVI.AS', 'EOSI.PA', 'ERA.PA', 'ERYP.PA', 'ESG.AS', 'ESGT.AS', 'ESI.PA', 'ESP.AS', 'ELNV.PA', 'EL.PA', 'ES.PA', 'EFI.PA', 'RF.PA', 'EUR.PA', 'ECT.AS', 'ECMPA.AS', 'ERFNV.PA', 'ERF.PA', 'EURN.BR', 'ENX.PA', 'ECP.PA', 'EUCAR.PA', 'ETL.PA', 'EGR.PA', 'EVS.BR', 'EXAC.PA', 'EXE.PA', 'EXM.BR', 'EPCP.PA', 'EXPL.PA', 'FAGR.BR', 'FAST.AS', 'EO.PA', 'FAUV.PA', 'FAYE.PA', 'FDJ.PA', 'FCMC.PA', 'FALG.PA', 'SACI.PA', 'ORIA.PA', 'FGA.PA', 'BERR.PA', 'EBPF.PA', 'FOAF.PA', 'FINM.PA', 'ODET.PA', 'FNTS.PA', 'FIPP.PA', 'FLOB.BR', 'FLOW.AS', 'FLUX.BR', 'FNAC.PA', 'FNG.BR', 'FPN.PA', 'LEBL.PA', 'FATL.PA', 'EURS.PA', 'INEA.PA', 'FLY.PA', 'SPEL.PA', 'FORE.PA', 'FFARM.AS', 'FOU.BR', 'LFDE.PA', 'FREY.PA', 'FUR.AS', 'GLPG.AS', 'GALIM.PA', 'GAM.PA', 'GBLB.BR', 'GEA.PA', 'GFC.PA', 'GNE.PA', 'GENX.PA', 'GNRO.PA', 'GNFT.PA', 'GKTX.PA', 'GV.PA', 'SIGHT.PA', 'GOJXN.AS', 'GET.PA', 'GIMB.BR', 'GLO.PA', 'GLOG.BR', 'GPE.PA', 'GRVO.PA', 'GVNV.AS', 'GREEN.BR', 'CEN.PA', 'FLO.PA', 'GOE.PA', 'IRD.PA', 'GJAJ.PA', 'OPN.PA', 'PARP.PA', 'SFPI.PA', 'GTBP.PA', 'GTT.PA', 'GBT.PA', 'GBTNV.PA', 'GUI.PA', 'HAL.AS', 'HAMO.BR', 'PIG.PA', 'HEIJM.AS', 'HEIA.AS', 'HEIO.AS', 'RMS.PA', 'HEXA.PA', 'HF.PA', 'HCO.PA', 'HIPAY.PA', 'HOLCO.AS', 'HOMI.BR', 'HOP.PA', 'HDP.PA', 'HDG.AS', 'HUNDP.AS', 'HYDRA.AS', 'HYL.BR', 'IBAB.BR', 'ICAD.PA', 'ICT.AS', 'IDL.PA', 'IDIP.PA', 'IEP.BR', 'IEX.AS', 'IGE.PA', 'ILD.PA', 'IMCD.AS', 'NK.PA', 'IMMOU.BR', 'IMDA.PA', 'IMMO.BR', 'INCLU.BR', 'INFE.PA', 'INF.PA', 'INGA.AS', 'IPH.PA', 'INPST.AS', 'ITP.PA', 'INTER.AS', 'INTO.BR', 'ITXT.PA', 'IVA.PA', 'IPN.PA', 'IPS.PA', 'IPSNV.PA', 'ITE.PA', 'JBOG.PA', 'JCQ.PA', 'DEC.PA', 'JDEP.AS', 'JEN.BR', 'TKWY.AS', 'KOF.PA', 'KBC.BR', 'KBCA.BR', 'KENDR.AS', 'KER.PA', 'KEYW.BR', 'KDS.AS', 'KIN.BR', 'LI.PA', 'KORI.PA', 'KPN.AS', 'OR.PA', 'ORNV.PA', 'LACR.PA', 'LHN.PA', 'MMB.PA', 'LAT.PA', 'LPE.PA', 'LVIDE.AS', 'LOUP.PA', 'LEAS.BR', 'LBON.PA', 'LSS.PA', 'LSSNV.PA', 'LR.PA', 'LIN.PA', 'FII.PA', 'LNA.PA', 'LOTB.BR', 'BOLS.AS', 'LBIRD.PA', 'MC.PA', 'LYS.PA', 'MRM.PA', 'MDM.PA', 'MALT.PA', 'MTU.PA', 'MAN.PA', 'MAREL.AS', 'IAM.PA', 'MKEA.PA', 'MAU.PA', 'MBWS.PA', 'MCPHY.PA', 'MDXH.BR', 'EDI.PA', 'MEDCL.PA', 'MELE.BR', 'MEMS.PA', 'MERY.PA', 'MRK.PA', 'MRN.PA', 'MRNNV.PA', 'METEX.PA', 'MMT.PA', 'ML.PA', 'MLNV.PA', 'MUN.PA', 'MIKO.BR', 'MITRA.BR', 'NEDSE.AS', 'MONC.PA', 'MONT.BR', 'MOP.BR', 'MOPF.BR', 'MORE.AS', 'MOUR.BR', 'GREV.PA', 'NACON.PA', 'NANO.PA', 'KN.PA', 'NAVYA.PA', 'NEDAP.AS', 'NEOEN.PA', 'NRP.AS', 'NRO.PA', 'NSE.AS', 'NEWAY.AS', 'NEX.PA', 'NXI.PA', 'NEXTS.PA', 'COX.PA', 'NN.AS', 'NOKIA.PA', 'NOVI.AS', 'NR21.PA', 'NRG.PA', 'NSI.AS', 'NYR.BR', 'NYXH.BR', 'OCI.AS', 'SBT.PA', 'OLG.PA', 'ONTEX.BR', 'ORA.PA', 'OBEL.BR', 'ORANW.AS', 'ORAP.PA', 'KAZI.PA', 'ORDI.AS', 'OREGE.PA', 'ORP.PA', 'OSE.PA', 'OXUR.BR', 'PID.PA', 'PAR.PA', 'PARRO.PA', 'PSAT.PA', 'PAT.PA', 'PAY.BR', 'PCA.PA', 'RI.PA', 'PERR.PA', 'PSH.AS', 'PEUG.PA', 'PHA.PA', 'PHARM.AS', 'INPHI.AS', 'PHIA.AS', 'PIC.BR', 'VAC.PA', 'PVL.PA', 'POM.PA', 'PORF.AS', 'PNL.AS', 'POXEL.PA', 'PPLAA.AS', 'PPLAB.AS', 'PPLA.AS', 'PREC.PA', 'PROAC.PA', 'PWG.PA', 'PROL.PA', 'PRX.AS', 'PROX.BR', 'PSB.PA', 'PUB.PA', 'QRF.BR', 'QDT.PA', 'QFG.BR', 'RAL.PA', 'GDS.PA', 'RAND.AS', 'REC.BR', 'RX.PA', 'REINA.AS', 'REN.AS', 'RCO.PA', 'RNO.PA', 'RWI.AS', 'RES.BR', 'RET.BR', 'RXL.PA', 'RBT.PA', 'CBR.PA', 'CBE.PA', 'RBO.PA', 'ROOD.AS', 'ENGB.BR', 'ROTH.PA', 'ROU.BR', 'RDSA.AS', 'RDSB.AS', 'RUI.PA', 'SK.PA', 'SAF.PA', 'SGO.PA', 'SGONV.PA', 'SABE.PA', 'SAMS.PA', 'SAN.PA', 'SANNV.PA', 'DIM.PA', 'SAVE.PA', 'SBMO.AS', 'CBSM.PA', 'SCHD.BR', 'SLB.PA', 'SU.PA', 'SCR.PA', 'SCHP.PA', 'SGRO.PA', 'SLCO.PA', 'SELER.PA', 'SEQUA.BR', 'SEFER.PA', 'SESG.PA', 'SESL.PA', 'SRP.PA', 'SHUR.BR', 'SIFG.AS', 'GIRO.PA', 'LIGHT.AS', 'SII.PA', 'SIOE.BR', 'SIP.BR', 'SLIGR.AS', 'SMAR.BR', 'SMCP.PA', 'SMTPC.PA', 'SNOW.AS', 'SFCA.PA', 'GLE.PA', 'SW.PA', 'SEC.PA', 'SOF.BR', 'SOG.PA', 'SOI.PA', 'LOCAL.PA', 'S30.PA', 'SOLV.BR', 'SOLB.BR', 'SO.PA', 'SOP.PA', 'SPA.BR', 'SPIE.PA', 'SQI.PA', 'DPT.PA', 'STF.PA', 'STLA.PA', 'STRN.AS', 'STM.PA', 'SEV.PA', 'SWP.PA', 'SDG.PA', 'TKTT.PA', 'TAYN.PA', 'TCH.PA', 'TE.PA', 'FTI.PA', 'TNET.BR', 'TEP.PA', 'TVRB.PA', 'TERB.BR', 'TESB.BR', 'TES.PA', 'TFG.AS', 'TEXF.BR', 'TFI.PA', 'TFF.PA', 'HO.PA', 'HONV.PA', 'THEP.PA', 'TBIRD.AS', 'TIE.AS', 'TKO.PA', 'TINC.BR', 'TIPI.PA', 'TITC.BR', 'TWEKA.AS', 'TOM2.AS', 'FP.PA', 'EC.PA', 'TOUP.PA', 'EIFF.PA', 'TNG.PA', 'TRI.PA', 'TUB.BR', 'UBI.PA', 'UCB.BR', 'UMI.BR', 'URW.AS', 'UNBL.PA', 'UPG.BR', 'UNA.AS', 'UFF.PA', 'FPG.PA', 'VLK.AS', 'FR.PA', 'VK.PA', 'VLA.PA', 'VALUE.AS', 'PREVA.AS', 'VAN.BR', 'VASTN.AS', 'VASTB.BR', 'VIE.PA', 'VEON.AS', 'VRLA.PA', 'VMX.PA', 'VETO.PA', 'VGP.BR', 'VCT.PA', 'VDLO.PA', 'VIL.PA', 'RIN.PA', 'DG.PA', 'VIO.BR', 'VIRP.PA', 'SDT.PA', 'VIV.PA', 'VIVNV.PA', 'VVY.AS', 'VTA.AS', 'VLTSA.PA', 'VPK.AS', 'VRAP.PA', 'WEB.BR', 'WAVE.PA', 'WDP.BR', 'MF.PA', 'WHA.AS', 'WEHB.BR', 'WKL.AS', 'WLN.PA', 'XFAB.PA', 'XIL.PA', 'XIOR.BR', 'XPO.PA', 'YATRA.AS', 'CV.PA', 'ZENT.BR', '^FCHI', '^N100']

# CAC40
cac40 = ['EURUSD=X', '^FCHI', 'AI.PA', 'AIR.PA', 'ALO.PA', 'MT.PA', 'ATO.PA', 'CS.PA', 'BNP.PA', 'EN.PA', 'CAP.PA', 'CA.PA', 'ACA.PA', 'BN.PA', 'DSY.PA', 'ENGI.PA', 'EL.PA', 'RMS.PA', 'KER.PA', 'OR.PA', 'LR.PA', 'MC.PA', 'ML.PA', 'ORA.PA', 'RI.PA', 'PUB.PA', 'RNO.PA', 'SAF.PA', 'SGO.PA', 'SAN.PA', 'SU.PA', 'GLE.PA', 'STLA.PA', 'STM.PA', 'TEP.PA', 'HO.PA', 'FP.PA', 'URW.PA', 'VIE.PA', 'DG.PA', 'VIV.PA', 'WLN.PA']

# Selection of stocks & reference indices for testing
stocks_for_testing = ["EDF.PA", "ENGI.PA", "ABIO.PA", "ELEC.PA", "NEOEN.PA", "NSE.AS", "VLTSA.PA", "^FCHI", "^N100"]

small_test = ['SU.PA', 'VLTSA.PA']

# mystocks = euronext100
mystocks = euronext_pab
# mystocks = cac40
# mystocks = small_test

In [20]:
# ?yf.download

In [21]:
# Get all data
data = yf.download(
    tickers=mystocks,
    period="2y",
    interval="1d",
    threads=True,
)

[*********************100%***********************]  706 of 706 completed

12 Failed downloads:
- ENNV.PA: No data found, symbol may be delisted
- VIVNV.PA: No data found, symbol may be delisted
- LSSNV.PA: No data found, symbol may be delisted
- CDANV.PA: No data found, symbol may be delisted
- ELNV.PA: No data found, symbol may be delisted
- DSYNV.PA: No data found, symbol may be delisted
- ORNV.PA: No data found, symbol may be delisted
- EFICT.AS: No data found, symbol may be delisted
- SGONV.PA: No data found for this date range, symbol may be delisted
- DSC2S.AS: No data found, symbol may be delisted
- ESGT.AS: No data found, symbol may be delisted
- BOLNV.PA: No data found, symbol may be delisted


In [22]:
# Pull closing prices and volumes. Check docs for all parameters
# data = yf.download(
#     tickers=mystocks,
#     period="2y",
#     interval="1d",
#     threads=True,
# )[["Adj Close", "Volume"]]

In [23]:
# Save data to s3
_write_dataframe_to_csv_on_s3("euronext-stocks", data, "data.csv")

Writing 514 records to 'data.csv' in bucket 'euronext-stocks'.


In [24]:
# Store volumes in separate dataframe
df_volumes = data["Volume"]
df_volumes = df_volumes[mystocks]

In [25]:
df_volumes.shape

(514, 706)

In [26]:
df_volumes.tail()

,2CRSI.PA,2MX.PA,ASP.PA,AALB.AS,ABI.BR,AB.PA,ABCA.PA,ABEO.PA,ABNX.PA,ABVX.PA,...,WLN.PA,XFAB.PA,XIL.PA,XIOR.BR,XPO.PA,YATRA.AS,CV.PA,ZENT.BR,^FCHI,^N100
Date,,,,,,,,,,,,,,,,,,,,,
2021-05-17,14682.0,6560.0,23441.0,101674.0,1093442.0,24097.0,45527.0,2186.0,252009.0,30425.0,...,383247.0,121277.0,5075.0,11628.0,175.0,NaN,0.0,0.0,62538700.0,159643800.0
2021-05-18,18371.0,4794.0,8493.0,165672.0,1152485.0,91428.0,47881.0,3687.0,416173.0,41383.0,...,369424.0,107235.0,7611.0,11500.0,378.0,NaN,0.0,6.0,74129300.0,175986500.0
2021-05-19,31618.0,15985.0,6722.0,242237.0,1715732.0,46275.0,50902.0,2573.0,258622.0,19911.0,...,607858.0,251417.0,2082.0,16948.0,178.0,NaN,0.0,83.0,92757600.0,217219200.0
2021-05-20,14517.0,12817.0,4780.0,201537.0,1217693.0,25391.0,22125.0,1870.0,562827.0,23676.0,...,421998.0,161614.0,2363.0,15840.0,4770.0,NaN,0.0,250.0,68528400.0,157211900.0
2021-05-21,10828.0,20313.0,12176.0,240460.0,1451419.0,67988.0,29852.0,5577.0,921742.0,89564.0,...,522588.0,277763.0,3999.0,17547.0,784.0,NaN,0.0,102.0,75862000.0,182668200.0


In [27]:
# Save df_volumes to s3
_write_dataframe_to_csv_on_s3("euronext-stocks", df_volumes, "df_volumes.csv")

Writing 514 records to 'df_volumes.csv' in bucket 'euronext-stocks'.


In [28]:
# Store prices in separate dataframe
df_prices = data["Adj Close"]
df_prices = df_prices[mystocks]

In [29]:
df_prices.shape

(514, 706)

In [30]:
df_prices.head()

,2CRSI.PA,2MX.PA,ASP.PA,AALB.AS,ABI.BR,AB.PA,ABCA.PA,ABEO.PA,ABNX.PA,ABVX.PA,...,WLN.PA,XFAB.PA,XIL.PA,XIOR.BR,XPO.PA,YATRA.AS,CV.PA,ZENT.BR,^FCHI,^N100
Date,,,,,,,,,,,,,,,,,,,,,
2019-05-23,9.04,NaN,3.900327,28.260750,67.888748,4.335,5.543856,28.962732,0.3900,8.96,...,50.099998,4.750,35.700001,40.271324,273.466217,3.36,0.2,16.299999,5281.370117,1031.750000
2019-05-24,9.02,NaN,3.971241,28.326471,67.732155,4.220,5.570255,28.468489,0.3830,9.07,...,51.500000,4.895,36.250000,40.544350,273.466217,3.36,0.2,16.400000,5316.509766,1037.339966
2019-05-27,9.10,NaN,3.957059,28.495472,68.432228,4.200,5.631852,27.974243,0.3900,9.05,...,51.500000,4.810,35.650002,40.726368,273.466217,3.36,0.2,16.500000,5336.189941,1041.579956
2019-05-28,9.20,NaN,3.933420,28.457916,68.128250,3.975,5.631852,28.270792,0.3705,9.03,...,52.799999,4.965,36.799999,39.952747,283.446747,3.36,0.2,16.600000,5312.689941,1038.910034
2019-05-29,9.10,NaN,3.914510,27.894579,66.857071,4.680,5.570255,28.171942,0.3695,9.03,...,52.450001,4.875,35.700001,40.271324,287.438965,3.36,0.2,16.600000,5222.120117,1022.989990


In [31]:
# Save df_prices to s3
_write_dataframe_to_csv_on_s3("euronext-stocks", df_prices, "df_prices.csv")

Writing 514 records to 'df_prices.csv' in bucket 'euronext-stocks'.


### Get ticker metadata

In [32]:
mytickers = yf.Tickers(mystocks)

In [33]:
# Get metadata (inspect an example for all the types of metadata that can be captured)
ticker_info = {}
for ticker in list(mytickers.tickers):
    print(ticker)
    ticker_dets = []
    info = mytickers.tickers[ticker].info
    
    for md in metadata:
        try:
            ticker_dets.append(info[md])
        except:
            ticker_dets.append("Missing")
    ticker_info[ticker] = ticker_dets

2CRSI.PA
2MX.PA
ASP.PA
AALB.AS
ABI.BR
AB.PA
ABCA.PA
ABEO.PA
ABNX.PA
ABVX.PA
ABN.AS
ABO.BR
ACPH.BR
ACAN.PA
ACCEL.AS
ACCB.BR
AC.PA
ACNV.PA
AXS.AS
ACKB.BR
EOS.PA
ATI.PA
ALDV.PA
ALP.PA
ADOC.PA
ADP.PA
ADUX.PA
ADV.PA
ADVIC.PA
ADVI.PA
ADYEN.AS
AED.BR
AGN.AS
AKOM.PA
AGS.BR
AGFB.BR
AD.AS
AF.PA
AI.PA
AIR.PA
AJAX.AS
AKA.PA
AKW.PA
AKZA.AS
AAA.PA
ABIO.PA
ALD.PA
ALFEN.AS
CDANV.PA
CDA.PA
ALM.PA
ALO.PA
ALONV.PA
LTA.PA
ALTA.PA
AREIT.PA
ATE.PA
ALTUR.PA
ALX.AS
AMG.AS
AMPLI.PA
ACOMO.AS
AMUN.PA
APAM.AS
APM.PA
ARCAD.AS
MT.AS
ARG.PA
ARGX.BR
AKE.PA
ARTE.PA
PRC.PA
ARTO.PA
ASC.BR
ASIT.BR
ASM.AS
ASML.AS
ASRNL.AS
ASY.PA
ATA.PA
ATEME.PA
ATEB.BR
ATO.PA
ATRS.AS
AUB.PA
AUGR.PA
AURE.PA
AURS.PA
AVTX.AS
AVT.PA
CS.PA
CSNV.PA
AXW.PA
BSGR.AS
BCRA.PA
BAIN.PA
BALTA.BR
BALYO.PA
BAMNB.AS
BANI.BR
BUI.PA
BAR.BR
BFIT.AS
BASS.PA
BLC.PA
BESI.AS
BEFB.BR
BEKB.BR
FBEL.PA
BELR.BR
BELU.BR
BEN.PA
BBED.AS
BEVER.AS
BB.PA
BIG.PA
BCART.BR
BIM.PA
BLEE.PA
BNP.PA
BOI.PA
BOL.PA
BOLNV.PA
BON.PA
BOTHE.BR
BOOST.PA
BOKA.AS
BSD.PA
BGHL.AS
BGHS.AS
EN.

In [34]:
# ticker_info

In [35]:
# Save ticker info to s3

s3 = boto3.client("s3")
serializedMyData = pickle.dumps(ticker_info)
# s3.put_object(Bucket="euronext-stocks", Key="ticker_info", Body=serializedMyData)
s3.put_object(Bucket="euronext-stocks", Key="ticker_info_cac40", Body=serializedMyData)
# s3.put_object(Bucket="euronext-stocks", Key="ticker_info_test", Body=serializedMyData)

{'ResponseMetadata': {'RequestId': 'SY2MWBATPGMGX1JM',
  'HostId': 'wFDAU55foDSOr8YbP48J7jID363y3V4aEzuhGGIVCuVaVQebDw8S3C8In+suJhzPKWzlCa4k7o0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'wFDAU55foDSOr8YbP48J7jID363y3V4aEzuhGGIVCuVaVQebDw8S3C8In+suJhzPKWzlCa4k7o0=',
   'x-amz-request-id': 'SY2MWBATPGMGX1JM',
   'date': 'Sun, 23 May 2021 14:26:17 GMT',
   'etag': '"7851be28c931fd4aa2727244fc4cab45"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"7851be28c931fd4aa2727244fc4cab45"'}

In [36]:
# Get data required for ratio analysis

ratio_data = {}

for ticker in list(mytickers.tickers):
    print(ticker)
    ratio_dets = []
    info = mytickers.tickers[ticker].info
    cashflow_statement = mytickers.tickers[ticker].cashflow
    bs_statement = mytickers.tickers[ticker].balance_sheet
    pnl = mytickers.tickers[ticker].financials
    
    for sr in static_ratios:
        try:
            ratio_dets.append(info[sr])
        except:
            ratio_dets.append(np.nan)
    
    for cf in cashflow:
        try:
            ratio_dets.append(cashflow_statement[pd.to_datetime(prev_yr_end)].loc[cf])
        except:
            ratio_dets.append(np.nan)
    
    for bs in balsheet:
        try:
             ratio_dets.append(bs_statement[pd.to_datetime(prev_yr_end)].loc[bs])
        except:
            ratio_dets.append(np.nan)
    
    for fin in financials:
        try:
            ratio_dets.append(pnl[prev_yr_end].loc[fin])
        except:
            ratio_dets.append(np.nan)
    
    ratio_data[ticker] = ratio_dets

2CRSI.PA
2MX.PA
ASP.PA
AALB.AS
ABI.BR
AB.PA
ABCA.PA
ABEO.PA
ABNX.PA
ABVX.PA
ABN.AS
ABO.BR
ACPH.BR
ACAN.PA
ACCEL.AS
ACCB.BR
AC.PA
ACNV.PA
AXS.AS
ACKB.BR
EOS.PA
ATI.PA
ALDV.PA
ALP.PA
ADOC.PA
ADP.PA
ADUX.PA
ADV.PA
ADVIC.PA
ADVI.PA
ADYEN.AS
AED.BR
AGN.AS
AKOM.PA
AGS.BR
AGFB.BR
AD.AS
AF.PA
AI.PA
AIR.PA
AJAX.AS
AKA.PA
AKW.PA
AKZA.AS
AAA.PA
ABIO.PA
ALD.PA
ALFEN.AS
CDANV.PA
CDA.PA
ALM.PA
ALO.PA
ALONV.PA
LTA.PA
ALTA.PA
AREIT.PA
ATE.PA
ALTUR.PA
ALX.AS
AMG.AS
AMPLI.PA
ACOMO.AS
AMUN.PA
APAM.AS
APM.PA
ARCAD.AS
MT.AS
ARG.PA
ARGX.BR
AKE.PA
ARTE.PA
PRC.PA
ARTO.PA
ASC.BR
ASIT.BR
ASM.AS
ASML.AS
ASRNL.AS
ASY.PA
ATA.PA
ATEME.PA
ATEB.BR
ATO.PA
ATRS.AS
AUB.PA
AUGR.PA
AURE.PA
AURS.PA
AVTX.AS
AVT.PA
CS.PA
CSNV.PA
AXW.PA
BSGR.AS
BCRA.PA
BAIN.PA
BALTA.BR
BALYO.PA
BAMNB.AS
BANI.BR
BUI.PA
BAR.BR
BFIT.AS
BASS.PA
BLC.PA
BESI.AS
BEFB.BR
BEKB.BR
FBEL.PA
BELR.BR
BELU.BR
BEN.PA
BBED.AS
BEVER.AS
BB.PA
BIG.PA
BCART.BR
BIM.PA
BLEE.PA
BNP.PA
BOI.PA
BOL.PA
BOLNV.PA
BON.PA
BOTHE.BR
BOOST.PA
BOKA.AS
BSD.PA
BGHL.AS
BGHS.AS
EN.

In [37]:
# store ratio data in dataframe
ratio_data_cols = static_ratios + cashflow + balsheet + financials # field names in correct order to be used in creating dataframe
df_ratio_data = pd.DataFrame(ratio_data, index=ratio_data_cols) 

In [38]:
df_ratio_data

,2CRSI.PA,2MX.PA,ASP.PA,AALB.AS,ABI.BR,AB.PA,ABCA.PA,ABEO.PA,ABNX.PA,ABVX.PA,...,WLN.PA,XFAB.PA,XIL.PA,XIOR.BR,XPO.PA,YATRA.AS,CV.PA,ZENT.BR,^FCHI,^N100
profitMargins,-5.605000e-02,0.0,3.096000e-02,4.494000e-02,2.997000e-02,0.000000e+00,5.078000e-01,-1.010000e-02,0.00,0.000000e+00,...,5.957000e-02,6.530000e-02,1.910200e-01,-3.300700e-01,7.590000e-03,NaN,NaN,9.743000e-02,NaN,NaN
payoutRatio,0.000000e+00,NaN,0.000000e+00,7.547000e-01,NaN,0.000000e+00,9.180000e-01,0.000000e+00,0.00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,NaN,0.000000e+00,NaN,NaN
sharesOutstanding,1.424340e+07,25000000.0,1.289690e+07,1.105800e+08,1.972250e+09,4.435370e+07,5.815140e+07,7.506850e+06,24410700.00,1.439940e+07,...,2.792050e+08,1.306320e+08,4.853540e+06,2.148290e+07,9.791790e+06,NaN,NaN,3.310880e+06,NaN,NaN
trailingEps,-2.600000e-01,NaN,4.200000e-01,1.060000e+00,5.790000e-01,-4.450000e-01,6.040000e-01,9.200000e-02,0.09,-2.611000e+00,...,8.140000e-01,2.100000e-01,1.268000e+00,-1.176000e+00,4.690000e+00,NaN,NaN,2.147000e+00,NaN,NaN
forwardEps,NaN,NaN,4.500000e-01,2.390000e+00,4.800000e+00,-1.000000e+00,5.900000e-01,1.470000e+00,NaN,-4.700000e-01,...,1.210000e+00,3.800000e-01,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total Cash From Operating Activities,NaN,NaN,2.215000e+07,3.776000e+08,1.089100e+10,-1.351100e+07,2.433300e+07,NaN,-600000.00,NaN,...,5.113000e+08,7.434600e+07,NaN,3.292000e+06,6.801970e+08,NaN,NaN,7.400000e+06,NaN,NaN
Total Liab,NaN,NaN,1.091440e+08,1.448900e+09,1.480590e+11,4.923700e+07,1.902600e+07,NaN,NaN,NaN,...,9.283600e+09,1.731200e+08,NaN,9.608110e+08,3.638293e+09,NaN,NaN,2.329100e+07,NaN,NaN
Total Stockholder Equity,NaN,NaN,4.066700e+07,1.774100e+09,6.802400e+10,-1.954900e+07,1.540310e+08,NaN,NaN,NaN,...,9.480600e+09,6.593340e+08,NaN,6.411960e+08,9.801930e+08,NaN,NaN,3.790800e+07,NaN,NaN
Total Current Assets,NaN,NaN,1.021530e+08,9.921000e+08,2.651900e+10,2.632500e+07,1.698700e+08,NaN,NaN,NaN,...,4.606100e+09,4.522080e+08,NaN,5.493400e+07,1.854354e+09,NaN,NaN,5.290500e+07,NaN,NaN
Total Current Liabilities,NaN,NaN,8.200400e+07,8.614000e+08,3.235200e+10,2.258700e+07,1.787900e+07,NaN,NaN,NaN,...,4.523100e+09,1.243360e+08,NaN,1.266180e+08,2.052550e+09,NaN,NaN,2.116000e+07,NaN,NaN


In [39]:
# Save df_ratio_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_ratio_data, "df_ratio_data.csv")

Writing 11 records to 'df_ratio_data.csv' in bucket 'euronext-stocks'.


### Pre-compute ratios and store to s3

#### Trailing PE

In [ ]:
# Table of Trailing P/E Ratios
trailing_pe_data = {}
for ticker in df_prices.columns:
    trailing_pe_data[ticker] = df_prices[ticker] / df_ratio_data[ticker].loc["trailingEps"]
df_trailing_pe_data = pd.DataFrame(trailing_pe_data)

In [ ]:
# Save df_trailing_pe_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_trailing_pe_data, "df_trailing_pe_data.csv")

#### Forward PE

In [ ]:
# Table of Forward P/E Ratios
forward_pe_data = {}
for ticker in df_prices.columns:
    forward_pe_data[ticker] = df_prices[ticker] / df_ratio_data[ticker].loc["forwardEps"]
df_forward_pe_data = pd.DataFrame(forward_pe_data)

In [ ]:
# Save df_forward_pe_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_forward_pe_data, "df_forward_pe_data.csv")

#### Price to Sales

In [ ]:
# Table of Price to Sales Ratios
p2s_data = {}
for ticker in df_prices.columns:
    p2s_data[ticker] = df_prices[ticker] / (df_ratio_data[ticker].loc["Total Revenue"] / df_ratio_data[ticker].loc["sharesOutstanding"])
df_p2s_data = pd.DataFrame(p2s_data)

In [ ]:
# Save df_p2s_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_p2s_data, "df_p2s_data.csv")

#### Profit Margin

In [ ]:
su = yf.Ticker("SU.PA")
# su.financials # [pd.to_datetime('2020-12-31')].loc["Total Liab"]
su.financials.loc["Net Income Applicable To Common Shares"] /su.financials.loc["Total Revenue"]

In [ ]:
# Table of Profit Margins
pm_data = {}
for ticker in df_prices.columns:
    dets = yf.Ticker(ticker)
    try:
        rev = dets.financials.loc["Total Revenue"]
    except:
        rev = np.nan
    try:
        profit = dets.financials.loc["Net Income Applicable To Common Shares"]
    except:
        profit = np.nan
    try:
        pm = profit/rev
    except:
        pm = np.nan
    pm_data[ticker] = pm
df_pm_data = pd.DataFrame(pm_data)

In [ ]:
# Save df_pm_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_pm_data, "df_pm_data.csv")

#### Dividend payout ratio

In [ ]:
# Table of Dividend payout ratios
dpr_data = {}
for ticker in df_prices.columns:
    dpr_data[ticker] = df_ratio_data[ticker].loc["payoutRatio"]
series_dpr_data = pd.Series(dpr_data)

In [ ]:
# Plot dividend payout ratios
# series_dpr_data.sort_values().plot.barh(figsize=(12,10))
# plt.grid()

In [ ]:
# Save series_dpr_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", series_dpr_data, "series_dpr_data.csv")

#### Price to Free Cashflow ratio

In [40]:
# Table of Price to Free Cashflow ratios
pfcf_data = {}
for ticker in df_prices.columns:
#     print(ticker)
    dets = yf.Ticker(ticker)
    try:
        coa = dets.cashflow[prev_yr_end].loc["Total Cash From Operating Activities"]
        capex = dets.cashflow[prev_yr_end].loc["Capital Expenditures"]
        ocf = coa + capex 
    except:
        ocf = np.nan
#     print(ocf)
    try:
        sos = df_ratio_data[ticker].loc["sharesOutstanding"]
    except:
        sos = np.nan
#     print(sos)
    try:
        ocfs = ocf / sos
    except:
        ocfs = np.nan
#     print(ocfs)
#     print(df_prices[ticker])
    pfcf_data[ticker] = df_prices[ticker] / ocfs
df_pfcf_data = pd.DataFrame(pfcf_data)

In [41]:
# Save df_pfcf_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_pfcf_data, "df_pfcf_data.csv")

Writing 514 records to 'df_pfcf_data.csv' in bucket 'euronext-stocks'.


#### Debt to Equity ratio

In [42]:
# Table of Debt Equity ratios
de_data = {}
for ticker in df_prices.columns:
#     print(ticker)
    dets = yf.Ticker(ticker)
    try:
        tot_liab = dets.balance_sheet.loc["Total Liab"]
    except:
        tot_liab = np.nan
#     print(tot_liab)
    try:
        tot_se = dets.balance_sheet.loc["Total Stockholder Equity"]
    except:
        tot_se = np.nan
#     print(tot_se)
    de_ratio = tot_liab/tot_se
#     print(de_ratio)
    de_data[ticker] = de_ratio
df_de_data = pd.DataFrame(de_data)

In [43]:
# Save df_de_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_de_data, "df_de_data.csv")

Writing 64 records to 'df_de_data.csv' in bucket 'euronext-stocks'.


#### Current ratio

In [44]:
# Table of Current ratios
cr_data = {}
for ticker in df_prices.columns:
#     print(ticker)
    dets = yf.Ticker(ticker)
    try:
        ca = dets.balance_sheet.loc["Total Current Assets"]
    except:
        ca = np.nan
#     print(ca)
    try:
        cl = dets.balance_sheet.loc["Total Current Liabilities"]
    except:
        cl = np.nan
#     print(cl)
    current_ratio = ca/cl
#     print(current_ratio)
    cr_data[ticker] = current_ratio
df_cr_data = pd.DataFrame(cr_data)

In [45]:
# Save df_cr_data to s3 (NB: THIS IS THE MOST RECENT PREVIOUS YEAR END DATA FROM FINANCIAL STATEMENTS)
_write_dataframe_to_csv_on_s3("euronext-stocks", df_cr_data, "df_cr_data.csv")

Writing 64 records to 'df_cr_data.csv' in bucket 'euronext-stocks'.
